In [2]:
from datasets import load_from_disk
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

In [3]:
llm = LLM(model="iamjoon/qwen2-7b-rag-ko-checkpoint-141")

INFO 01-08 06:25:30 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='iamjoon/qwen2-7b-rag-ko-checkpoint-141', speculative_config=None, tokenizer='iamjoon/qwen2-7b-rag-ko-checkpoint-141', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=iamjoon/qwen2-7b-rag-ko-checkpoint-141, use_v2_block_manager=False, num_scheduler_steps=1, enabl

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 01-08 06:25:37 model_runner.py:926] Loading model weights took 14.2487 GB
INFO 01-08 06:25:40 gpu_executor.py:122] # GPU blocks: 61092, # CPU blocks: 4681
INFO 01-08 06:25:46 model_runner.py:1217] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 01-08 06:25:46 model_runner.py:1221] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 01-08 06:26:01 model_runner.py:1335] Graph capturing finished in 16 secs.


In [4]:
test_dataset = load_from_disk('test_dataset')

In [5]:
model = 'qwen2-7b-rag-ko/checkpoint-141'
tokenizer = AutoTokenizer.from_pretrained(model)

prompt_lst = []
label_lst = []

for prompt in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(
        prompt, tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1]
    prompt_lst.append(input)
    label_lst.append(label)

In [6]:
# 샘플링 파라미터 설정
sampling_params = SamplingParams(
    temperature=0,
    max_tokens=1024,
    stop=["<|im_end|>", "<|eot_id|>"],  # 문자열로 중지 토큰 지정
)

# 생성
outputs = llm.generate(prompt_lst[0], sampling_params)

# 결과 출력
for output in outputs:
    print()
    print(output.outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.61s/it, est. speed input: 2248.14 toks/s, output: 72.64 toks/s]



북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 대략 한 달입니다. 장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보이며, 이후 한 달간 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망입니다 [[ref2]].


In [ ]:
outputs = llm.generate(prompt_lst, sampling_params)

Processed prompts:  30%|██▉       | 901/3008 [04:46<10:58,  3.20it/s, est. speed input: 12110.95 toks/s, output: 237.71 toks/s]